In [55]:
import requests

from bs4 import BeautifulSoup

import sqlite3

In [56]:
def print_prices(list_prices):
    for trs in list_prices:
        name = trs.find("th").get_text().strip()
        tds = trs.find_all("td")
        print(f"{tds[4].get_text().strip()}  |  {tds[0].get_text().strip()} (ريال) : {name}")

In [57]:
insert_sql = """INSERT INTO Prices VALUES(?, ?, ?, ?)"""

In [58]:
def insert(list_prices):
    global insert_sql
    global curs
    global conn
    for trs in list_prices:
        name = trs.find("th").get_text().strip()
        tds = trs.find_all("td")
        prices = tds[0].get_text().strip()
        date = tds[4].get_text().strip()
        curs.execute(insert_sql, [name, "ريال", prices, date])
        conn.commit()

In [59]:
conn = sqlite3.connect("prices.db")

curs = conn.cursor()

In [60]:
update1 = "UPDATE Prices SET قیمت = ? WHERE عنوان = ?"
update2 = "UPDATE Prices SET زمان = ? WHERE عنوان = ?"

In [61]:
def update(list_prices):
    global update1
    global update2
    global curs
    global conn
    for trs in list_prices:
        name = trs.find("th").get_text().strip()
        tds = trs.find_all("td")
        prices = tds[0].get_text().strip().replace(",", "")
        int_price = int(prices)
        date = tds[4].get_text().strip()
        curs.execute(update1, [int_price, name])
        curs.execute(update2, [date, name])
        conn.commit()

In [62]:
def update_all(a):
    for list_all_prices in a:
        update(list_all_prices)

In [63]:
def print_all(a):
    for list_all_prices in a:
        print_prices(list_all_prices)

In [64]:
response = requests.get("https://www.tgju.org/")

soup = BeautifulSoup(response.text, "html.parser")

In [65]:
prices_div = soup.find_all("div", {"class" : "fs-cell fs-sm-12 fs-xs-12 fs-md-12 fs-lg-12 fs-xl-6 table-tag-row"})

In [66]:
table = prices_div[1].find("table", {"class" : "data-table market-table dark-head mobile-half"})

In [67]:
gold_prices = table.find("tbody")

In [68]:
list_golds_prices = gold_prices.find_all("tr")

In [69]:
coin_div = prices_div[6]

In [70]:
coin_div = prices_div[6]

In [71]:
coin_table = coin_div.find("table", {"class" : "data-table market-table dark-head market-section-right"})

In [72]:
coin_prices = coin_table.find("tbody").find_all("tr")

In [73]:
free_currency_div = soup.find("div", {"class" : "col-12 col-lg-12 col-xl-6 index-tabs-data-col-1"})
table_free_currency = free_currency_div.find("table", {"class" : "dataTable index-tabs-table data-table market-section-right market-table mobile-half"})

In [74]:
free_currency_prices = table_free_currency.find("tbody").find_all("tr")

In [75]:
Prices_Of_All = [list_golds_prices, coin_prices, free_currency_prices]

In [76]:
update_all(Prices_Of_All)